In [5]:
using DataFrames #NB: we are using pinned version 0.9.1 to match JuliaBox
using FreqTables
using Plots
using StatPlots
using Distributions
using DecisionTree

gr()
#Plots.scalefontsizes(1.5)

Plots.GRBackend()

# IL027 Core Lecture 3 part b - Data Analysis

### James Kermode

### School of Engineering

## Overview

- Reading datasets
- Visualisation
- Clustering and classification
- Missing data
- Feature engineering

# Iris Dataset

## Loading Data and Initial Exploration

We start with the classic [Iris flower dataset](https://en.wikipedia.org/wiki/Iris_flower_data_set), from a 1936 paper by Ronald Fisher. We can load this from the `"iris.csv"` file provived.

In [14]:
iris = readtable("iris.csv");

In [15]:
?iris

search: clibraries InsertionSort InverseWishart DiscreteDistribution



No documentation found.

`iris` is of type `DataFrames.DataFrame`.

**Summary:**

```
mutable struct DataFrames.DataFrame <: DataFrames.AbstractDataFrame
```

**Fields:**

```
columns  :: Array{T,1} where T
colindex :: DataFrames.Index
```


In [16]:
head(iris) # print the first few rows

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
1,5.1,3.5,1.4,0.2,setosa
2,4.9,3.0,1.4,0.2,setosa
3,4.7,3.2,1.3,0.2,setosa
4,4.6,3.1,1.5,0.2,setosa
5,5.0,3.6,1.4,0.2,setosa
6,5.4,3.9,1.7,0.4,setosa


### Size and Shape of Data

The `DataFrames` packages provides a number of functions to describe the data.

In [17]:
@show size(iris)
@show nrow(iris)
@show ncol(iris)
@show names(iris)
@show eltypes(iris);

size(iris) = (150, 5)
nrow(iris) = 150
ncol(iris) = 5
names(iris) = Symbol[:SepalLength, :SepalWidth, :PetalLength, :PetalWidth, :Species]
eltypes(iris) = Type[Float64, Float64, Float64, Float64, String]


In [18]:
describe(iris[:Species])

Summary Stats:
Length:         150
Type:           String
Number Unique:  3
Number Missing: 0
% Missing:      0.000000


We see there are three possible values for the species in this dataset. The `levels()` function identifies what these are:

In [19]:
species = levels(iris[:Species])

3-element DataArrays.DataArray{String,1}:
 "setosa"    
 "versicolor"
 "virginica" 

## Visualisation

### Scatter Plots

We start with a 2D scatter plot, to show the relationship between two variables, e.g. sepal width and sepal length

In [20]:
scatter(iris[:SepalWidth], iris[:SepalLength], group=iris[:Species])

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2.0 
 
 
 2.5 
 
 
 3.0 
 
 
 3.5 
 
 
 4.0 
 
 
 5 
 
 
 6 
 
 
 7 
 
 
 8 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 setosa 
 
 
 
 
 versicolor 
 
 
 
 
 virginica

There is a special syntax for plotting dataframes which saves a bit of typing, e.g. to look now at petal width vs. petal length. This uses the `@df` macro (similar to `@show` that we saw earlier).

In [21]:
@df iris scatter(:PetalWidth, :PetalLength, group=:Species)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.5 
 
 
 1.0 
 
 
 1.5 
 
 
 2.0 
 
 
 2.5 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 5 
 
 
 6 
 
 
 7 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 setosa 
 
 
 
 
 versicolor 
 
 
 
 
 virginica

### Histogram

Histograms are useful to visualise the distribution of individual variables

In [22]:
p1 = @df iris histogram(:PetalLength, bins=30, label="PetalLength", c=1)
p2 = @df iris histogram(:PetalWidth, bins=30, label="PetalWidth", c=2)

plot(p1, p2)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 5 
 
 
 6 
 
 
 7 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 15 
 
 
 20 
 
 
 25 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 PetalLength 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.5 
 
 
 1.0 
 
 
 1.5 
 
 
 2.0 
 
 
 2.5 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 15 
 
 
 20 
 
 
 25 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 PetalWidth

### Marginal histograms

Marginal histograms allow the correlation between two variables to be assessed (ignore warning message from `GR` backend)

In [23]:
@df iris marginalhist(:PetalLength, :PetalWidth, bins=30)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 5 
 
 
 6 
 
 
 7 
 
 
 0.5 
 
 
 1.0 
 
 
 1.5 
 
 
 2.0 
 
 
 2.5 
 
 
<image width="442" height="292" xlink:href="data:;base64,
iVBORw0KGgoAAAANSUhEUgAAAboAAAEkCAYAAABZm/S2AAAGEUlEQVR4nO3coW5kZRyH4WkzZoIZ
gWhC1pAgGlKP4DK4CsQILoCLqMDg0NwD2WtoSAUYglk5iM2YskWQYFn+h+30vH0e/+Wc0+72zWd+
FxsA/gfbx/nZh4vpyf3uZvzc4+lu/Nw1uTz3CwDAhyR0AKQJHQBpQgdAmtABkCZ0AKQJHQBpQgdA
mtABkCZ0AKQJHQBpL2LnDFij82xHTtmcfL7c6ABIEzoA0oQOgDShAyBN6ABIEzoA0oQOgDShAyBN
6ABIEzoA0oQOgDT7asAzNd+63O+ux089nu7HZ9fn6TdBl5n9m3CjAyBN6ABIEzoA0oQOgDShAyBN
6ABIEzoA0oQOgDShAyBN6ABIEzoA0la2cwbwXM23OZd5GXuVf5t9qxsdAGlCB0Ca0AGQJnQApAkd
AGlCB0Ca0AGQJnQApAkdAGlCB0Da9twvAPCc7Hc3Tz7ldTzdP/UjXxQ3OgDShA6ANKEDIE3oAEgT
OgDShA6ANKEDIE3oAEgTOgDShA6ANKEDIO3i3C8AlG3Hu5H73fX4qcu2Ix/8XfxX89/rMrPfjRsd
AGlCB0Ca0AGQJnQApAkdAGlCB0Ca0AGQJnQApAkdAGlCB0Ca0AGQtj33CwBl893IV5efj/cUj5sl
W5drs2R3cvr7WdceqBsdAGlCB0Ca0AGQJnQApAkdAGlCB0Ca0AGQJnQApAkdAGlCB0DaqmZcgHU5
XB3G81S3b75b8OT5RNV+dzN65+PpJc2OmQADgGdD6ABIEzoA0oQOgDShAyBN6ABIEzoA0oQOgDSh
AyBN6ABIEzoA0la1VwYssR3vTp5n23Bt78tz5UYHQJrQAZAmdACkCR0AaUIHQJrQAZAmdACkCR0A
aUIHQJrQAZAmdACkbc/9AsAaTHcnbU6+H7ueH5IbHQBpQgdAmtABkCZ0AKQJHQBpQgdAmtABkCZ0
AKQJHQBpQgdAmukYWJX5VNTh6uvxU3/446fx2XM4nu78beMfbnQApAkdAGlCB0Ca0AGQJnQApAkd
AGlCB0Ca0AGQJnQApAkdAGlCB0CaPTgYm+5OPpzl/91+dzPeyXx1+fno3N3bH6eP3Jzr5zQ33yFd
37euixsdAGlCB0Ca0AGQJnQApAkdAGlCB0Ca0AGQJnQApAkdAGlCB0Ca0AGQZl+NF+7p9wkPV4fx
M397+256dPP64fX47PF0Pzxpw5Hzc6MDIE3oAEgTOgDShA6ANKEDIE3oAEgTOgDShA6ANKEDIE3o
AEgzzwNjS+bDXgoTYJyfGx0AaUIHQJrQAZAmdACkCR0AaUIHQJrQAZAmdACkCR0AaUIHQJrQAZC2
PfcLULNk/3Fdu4j73fXo3PF0t6rvhLVzowMgTegASBM6ANKEDoA0oQMgTegASBM6ANKEDoA0oQMg
TegASBM6ANJs7vFs7Hc3C3YyZ46n+wWnZ9ucS77TTib8d250AKQJHQBpQgdAmtABkCZ0AKQJHQBp
QgdAmtABkCZ0AKQJHQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAABVF+d+gZnt4+zcw0q/F4Cpy3O/AAB8SEIHQJrQAZAmdACkCR0AaUIH
QJrQAZAmdACkCR0AaUIHQJrQAZAmdACkCR0AaUIHQJrQAZAmdACkCR0AaUIHQJrQAZAmdACkCR0A
aUIHQJrQAZB2sez49nF68uajr8ZP/f3dz6Nzx9P9+JmbzcPCnxUA5+BGB0Ca0AGQJnQApAkdAGlC
B0Ca0AGQJnQApAkdAGlCB0Ca0AGQtl1yeL+7Hp/99PLj8dm7t9MpLzNeAC+NGx0AaUIHQJrQAZAm
dACkCR0AaUIHQJrQAZAmdACkCR0AaUIHQJrQAZC23Wy2j9PD337y5fjBh1++GJ+9/ezP0blvfv1+
/K12MgHWyY0OgDShAyBN6ABIEzoA0oQOgDShAyBN6ABIEzoA0oQOgDShAyBN6ABIu1iydblk//Fw
dRg/9/bNrd1JAN6LGx0AaUIHQJrQAZAmdACkCR0AaUIHQJrQAZAmdACkCR0AaUIHQNpf/mSGFXQW
qTQAAAAASUVORK5CYII=
" transform="translate(45, 75)"/>

### Correlation plot

A correlation plot combines histograms for each variable (diagonal) with marginal histograms (above diagonal) and scatter plots for each pair (below diagonal)

In [24]:
@df iris corrplot([:SepalLength :SepalWidth :PetalLength :PetalWidth], bins=20, grid=true)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<image width="105" height="76" xlink:href="data:;base64,
iVBORw0KGgoAAAANSUhEUgAAAGkAAABMCAYAAABu45m/AAACPUlEQVR4nO2dIU8DQRBGW3KGIpq0
tpZfUFGLoHgEhhB8Bb4KQRBNkIhqGoJBIjEYBKKKH1BBbW0xTYrEzRyZ2+t+4T277N4lLzOT3Z0e
zQaUoNja45tmyqfvpVwcqgFJAiBJgKKmxzg53abfOjfH5+tZ0pqwa4gkAZAkAJIEyCSX73YfkjtE
kgBIEgBJAmSS62P7KB+vpuVdE4kkAZAkAJIEyKQmeURrlvY+i0gSAEkCIEmAmu6TPOya8zo4NWeP
P/fN8fl69vdXyggiSQAkCYAkASqqSWnPvo6u3+0/ODuOLJ89RJIASBIASQJUVJNi9zXePujtxl59
2Gmb4/O1PT93iCQBkCQAkgQoVZP6rcvQfc58/RSZ3hh+PJs1z3s/bzz3XnIiSQAkCYAkASrZJ3k5
3a9p3+bouHdlzp8s74N9dXlDJAmAJAGQJEAt+4PV6NCsCd3pwpzv/2bW24el7rtLe59GJAmAJAGQ
JEAl+yRvH3TyYM9fjezxuxfvvki75njrE0kCIEkAJAlQsibZOdPrMZgsp+Z4d+rl9EXo7M0/+7Pf
r8S3jZw3iNU0IkkAJAmAJAFqutvf7XcS/B6H6NkfZ3f/HiQJgCQBinru/9OebY179uHf7dfAHB8c
2E8fdtrBHosYRJIASBIASQJUkktT91pvto/m+kXzItT3Ry84hEGSAEgSIOtc/Etsn+TdF/nwDVZw
QJIASBKgZK7dbd9ZHL7BColBkgBIEkAkV+f9vyRSQyQJgCQBkCTAD8XWgtfryUl7AAAAAElFTkSu
QmCC
" transform="translate(168, 12)"/>
 
 
 
 
 
 
 
 
 
 
<image width="105" height="76" xlink:href="data:;base64,
iVBORw0KGgoAAAANSUhEUgAAAGkAAABMCAYAAABu45m/AAACN0lEQVR4nO2coUtDURSH3+Q1FwbG
LayYdMEoKIgoLOwfMJhF7LJisgy7iFXD/oE1y4LC2pL9BdcFn0VEDVa558Dervu99331Xu4O+3bu
4bydrZbAgkm/7T2ftdDqSlGhwOJAkgBIEiB4F0IR2DWp3zoNrpNJAiBJACQJQE1aOPRJlQBJAiBJ
gPS/A9DGrje9+ol5yii/Dq6TSQIgSQAkCVDhPsmqJ+HexXeG95wwZJIASBIASQJUtk9q1nesLWa9
2Uo2zNexeiAPZJIASBIASQKUsiY163uO/mV+Nhv2Z3yUz/86ZJIASBIASQKUsibN8kdzj9UnHTc6
5hmdxpsdzIu9xYJMEgBJAiBJgFLWJMdzueSq3Q6uD7Mv84z718wRDXN3lQBJAiBJgFLWJA/d3afg
+nnWNM/w9GPMOFQEJAmAJAHSWLNjxWHH65k9WLsJ15N+q2ueMcjHUd4XMkkAJAmAJAFcd6pnZmAW
6X7u1c/MWI7a7+Y5w2w1uD7Kbx3RxKnVZJIASBIASQKknjveM182yMdFxGMyTZ7tTZlnRtuqOcvT
G5JJAiBJACQJUCvwvwgi3eFqzxrnh0wSAEkCIEmAdHqwb27qPGzbB9Xm/22o7xmhZ66gXJBJAiBJ
ACQJkF5M1s1Nl4d3EUKJN8f2SxH/LRQHMkkAJAmAJAFSz3dFvclHhFCSJG4dWJ6aY0EmCYAkAZAk
gOv3SbFm6uBvyCQBkCQAkgRwzTgo9RRlhEwSAEkCIEmAH+MrZoiNCQROAAAAAElFTkSuQmCC
" transform="translate(318, 12)"/>
 
 
 
 
 
 
 
 
 
 
<image width="112" height="76" xlink:href="data:;base64,
iVBORw0KGgoAAAANSUhEUgAAAHAAAABMCAYAAAC1WDOdAAACVUlEQVR4nO3bvUoDQRQF4FkJCAli
GQVLmxDSKlgG8gKCTyAomKDvIkRIYWenRWqFdAYMdhKEVDYWW+sGhMBaWCpzLu5PcnbP1+6yO3qY
XO7sTOAkQ5XYf30RJH3DWtIHyHIpQHIKkFxl2QMosnptz3u9GbdAjXRuNB9466RmIDkFSE4Bkkvc
h5QX6vGcS6PPQzQDySlAcgqQnPrAP+H6hno855wLo3HmY9EMJKcAySlAciXtA7P/TpfXezQDySlA
cgqQXCn7QNTDpdO/6XugGChAcgqQXAFrIF7HbMYt7/XQTQzf+rAwmqD3wGfUawdaCy0yBUhOAZIr
YA3EpsELuAOvUbarp6nUSWQUqQ8sNAVITgGSK1wNtOxVSaMPbGysw/f0w0tQSy17b9QHFpoCJKcA
yRHWQMuZBL/DHf/16Tuuo3efz0mH4Sz9Jvo2qRlITgGSU4DkjDVwNc7CpaU3uwZ34L+lWz+H/5O+
zkYIogDJKUByxrqVvPdKq0aitcHHDn7G7hDtRclnrGE01tmIslOA5BQgOVMf2K2fwXteP76810fz
gW1ECZ3c+7/1/cD7MdOQRo1DNAPJKUByCpBcYOnx2tVj+CB0jm215HVGPnuageQUIDkFSM70W7+I
b2CdvNjy91b98MrypsRnErhqcXKageQUIDkFSC6w7O1obPrXOZ1zrjfLq/bk0cPx7AHSDCSnAMkp
QHKm3/GH/SNYEzpPtytRE8pGM5CcAiSnAMmZ9sQM37azHof8k2YgOQVITgGSM+2JWZV1P/lNM5Cc
AiSnAMl9A51KgP8uXi9uAAAAAElFTkSuQmCC
" transform="translate(465, 12)"/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<image width="105" height="72" xlink:href="data:;base64,
iVBORw0KGgoAAAANSUhEUgAAAGkAAABICAYAAAD1ctupAAACB0lEQVR4nO3cMUvDQByG8UQCLhU6
FR0cWhU6i9it4NDBSRy0FBychA4uCv0QfoGCuKqIszj0C6iIrhW0Gdzq4tBJKjq43/8kMebV57fm
SM8+HB6XtmHgJfqwx4xDv3vhuyZ+ewKwEUkAkQQQSQCRBITs3PKPlSSASAKIJIBIAogkwHN354Md
4E9hJQkgkgAiCSCSgMhnUK+2bo5pXJ8nnszflHxjxkoSQCQBRBJAJAFEEhCVCw1z0NHDdAZT+c/c
R2qsJAFEEkAkAUQSED63Fs1ji5mTPfNGUbgl9jwpredoluTP2VhJAogkgEgCiCSASAKi/mDOHFSf
Os5gKtk6qOw4r3eHsXmPdqlsjukOY3MXGY8uORZSRyQBRBJAJAFhr7Zh/mOrVp7MG82e3uXmWMjn
b6pvXji

## Clustering and Classification

Now we've played around with our data a little, let's try to do some more detailed analysis. We would like to learn the relationship between the four variables and the species of iris. 

We can do this using clustering (*unsupervised learning*, i.e. only the features without labels are used) or classification (*supervised learing*, i.e. we provide the labels for a training set).

**K-means clustering** is a classic method for clustering that produces a fixed number $K$ of clusters, based on solving the optimisation problem

$$
\mathrm{minimize} \ \| \mathbf{x}_i - \boldsymbol{\mu}_{z_i} \|^2
\mathrm{with\ respect\ to} \ (\boldsymbol{\mu}, z)
$$

where $\boldsymbol{\mu}_k$ is the center of the $k$-th cluster, and $z_i$ indicates the cluster for $\mathbf{x}_i$. The implementation is fairly straighforward; here is an unoptimised version, see also `Clustering` package for faster code.

In [25]:
function kmeans(data, K; means=nothing, update_means=true)
    N, M = size(data)
    if means == nothing
        # Initialise centers randomly within range of data        
        means = hcat([rand(Uniform(dmin, dmax), K) for (dmin, dmax) 
                        in zip(minimum(data,1), maximum(data,1)) ]...)
    end
    assign = zeros(Int, N)
    oldassign = zeros(Int, N)
    while true
       for n in 1:N  # E-step - update the assignment
           d = data[n, :] .- means' # distance from all the means
           dmin, kmin = findmin([norm(d[:, i]) for i=1:K])
           assign[n] = kmin # assign point to closest centre
       end
       all(oldassign == assign) && break # if nothing changed, we're done
       oldassign = copy(assign)
        
       if update_means
           means[:] = 0.0 # M-step - update the centers
           for k in 1:K
               any(assign .== k) && (means[k,:] = mean(data[assign .== k, :], 1))
           end
       end
   end
   return (assign, means)
end

kmeans (generic function with 1 method)

### Applying K-means to the Iris Dataset

We create features and labels from the first four and last columns of the dataset, respectively (ignore deprecation warning)

In [28]:
features = convert(Array, iris[:, 1:4])
labels = convert(Array, iris[:, :Species])

assign, means = kmeans(features, 3); # K=3 clusters

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] anyna(::DataArrays.DataArray{Float64,1}) at ./deprecated.jl:57
 [3] convert(::Type{Array{Float64,2}}, ::DataFrames.DataFrame) at /home/jrun/.julia/v0.6/DataFrames/src/abstractdataframe/abstractdataframe.jl:522
 [4] convert(::Type{Array}, ::DataFrames.DataFrame) at /home/jrun/.julia/v0.6/DataFrames/src/abstractdataframe/abstractdataframe.jl:508
 [5] include_string(::String, ::String) at ./loading.jl:522
 [6] include_string(::Module, ::String, ::String) at /home/jrun/.julia/v0.6/Compat/src/Compat.jl:174
 [7] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /home/jrun/.julia/v0.6/IJulia/src/execute_request.jl:154
 [8] (::Compat.#inner#16{Array{Any,1},IJulia.#execute_request,Tuple{ZMQ.Socket,IJulia.Msg}})() at /home/jrun/.julia/v0.6/Compat/src/Compat.jl:496
 [9] eventloop(::ZMQ.Socket) at /home/jrun/.julia/v0.6/IJulia/src/eventloop.jl:8
 [10] (::IJulia.##14#17)() at ./task.jl:335
while loading In[28], in expression start

We can compare the true assignment with the one we get from K-means using scatter plots

In [29]:
cols = [1, 3]
p1 = scatter(iris[:,cols[1]], iris[:,cols[2]], 
             group=labels, title="True Assignment", marker=:o,
             xlabel=names(iris)[cols[1]], ylabel=names(iris)[cols[2]])
p2 = scatter(iris[:,cols[1]], iris[:,cols[2]], group=assign, 
             title="K-means",
             xlabel=names(iris)[cols[1]], ylabel=names(iris)[cols[2]])
scatter!(means[:,cols[1]], means[:,cols[2]], ms=10, marker=:s,
         label="Centers", color=[:blue, :red, :green])
plot(p1, p2, layout=(1,2))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 5 
 
 
 6 
 
 
 7 
 
 
 8 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 5 
 
 
 6 
 
 
 7 
 
 
 True Assignment 
 
 
 SepalLength 
 
 
 PetalLength 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 setosa 
 
 
 
 
 versicolor 
 
 
 
 
 virginica 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 5 
 
 
 6 
 
 
 7 
 
 
 8 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 5 
 
 
 6 
 
 
 7 
 
 
 K-means 
 
 
 SepalLength 
 
 
 PetalLength 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1 
 
 
 
 
 2 
 
 
 
 
 3 
 
 
 
 
 Centers

### Confusion matrix

The confusion matrix gives the number of results in each class, with predictions on rows and true labels as columns. This means we can see at a glance how many assignments are correct (diagonal entries) and how many are wrong (off-diagonal). 

We're using the `confusion_matrix` function from `DataFrames`, but it would be easy to code up by hand. Since the clustering was unsupervised, we first have to match up the labels and assignments by hand:

In [31]:
function assign2predict(g)
    if g == 2
        return "setosa"
    elseif g == 3
        return "versicolor"
    elseif g == 1
        return "virginica"
    end
end

predictions = assign2predict.(assign)
cm = confusion_matrix(labels, predictions)

3×3 Array{Int64,2}:
 50   0   0
  0  47   3
  0  14  36

Classes:  String["setosa", "versicolor", "virginica"]
Matrix:   
Accuracy: 0.8866666666666667
Kappa:    0.8300000000000001

Accuracy is estimated from the number of correct predictions divided by the total number of predictions. Because the assignments are based on distance, it only works well for roughly spherical clusters.

In [32]:
accuracy = sum(diag(cm.matrix))/sum(cm.matrix) * 100.

88.66666666666667

## Decision Tree Classifier

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/c/c6/Manual_decision_tree.jpg/220px-Manual_decision_tree.jpg" align="right" width="25%">

This is based on the idea of a tree of decisions, which you may be familiar with from conditional probability.

Here, the target value can take a discrete set of values, so this is called a *classification tree*. Leaves represent labels and branches represent combinations of features that lead to those class labels. Decision trees can also be used for *regression*, that is where the target variable is continouous.

This is an example of a *supervised learning* appraoach. We will use the implementation in the `DecisionTree` package.
First we need to split our dataset into ~2/3 training and ~1/3 test sets:

In [33]:
train = rand(nrow(iris)) .< 2/3
test = .!train 
sum(train), sum(test)

(94, 56)

In [34]:
model = build_tree(labels[train], features[train,:])
print_tree(model, 4)

Feature 3, Threshold 3.3
L-> setosa : 32/32
R-> Feature 4, Threshold 1.8
    L-> Feature 3, Threshold 5.0
        L-> versicolor : 26/26
        R-> Feature 2, Threshold 2.7
            L-> virginica : 2/2
            R-> versicolor : 1/1
    R-> virginica : 33/33


We first check for consistency on the training set:

In [37]:
predictions = apply_tree(model, features[train,:])
cm = confusion_matrix([String(L) for L in labels[train]], predictions)

3×3 Array{Int64,2}:
 32   0   0
  0  27   0
  0   0  35

Classes:  String["setosa", "versicolor", "virginica"]
Matrix:   
Accuracy: 1.0
Kappa:    1.0

Now we apply to the test set:

In [38]:
predictions = apply_tree(model, features[test, :])
cm = confusion_matrix([String(L) for L in labels[test]], predictions)
@show cm
cm.matrix

3×3 Array{Int64,2}:
 18   0   0
  1  21   1
  0   3  12

cm = Classes:  String["setosa", "versicolor", "virginica"]
Matrix:   
Accuracy: 0.9107142857142857
Kappa:    0.8632144601856375


3×3 Array{Int64,2}:
 18   0   0
  1  21   1
  0   3  12

### Remark: supervised learning with K-means

We could adapt the K-means algorithm to do supervised learning by initially allowing the means to move when clustering from the training data, and then fixing the means before predicting on the test data. 

In [39]:
assign_train, means = kmeans(features[train,:], 3, means=means, update_means=true) 
assign_test, means = kmeans(features[test,:], 3, means=means, update_means=false)
predict_test = assign2predict.(assign_test)
cm = confusion_matrix([String(L) for L in labels[test]], predictions)
@show cm
cm.matrix

3×3 Array{Int64,2}:
 18   0   0
  1  21   1
  0   3  12

cm = Classes:  String["setosa", "versicolor", "virginica"]
Matrix:   
Accuracy: 0.9107142857142857
Kappa:    0.8632144601856375


3×3 Array{Int64,2}:
 18   0   0
  1  21   1
  0   3  12

K-means is suprisingly good on this dataset for such a simple algorithm. However, note we had to choose the number of clusters and map the clusters to labels by hand.

# Titanic Dataset

We now move onto a more challenging dataset, taken from the passenger records for the *RMS Titanic*, which sank on the 14th April 1912 with the loss of more than 1500 lives. The task is to use data such as the age, sex and fare paid by a passenger to predict whether they survived or not. As before, we will use a portion of the dataset to train a predictive model, and then assess it using the remaining test data.

In [40]:
titanic = readtable("titanic.csv")
head(titanic)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,NA,S
2,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1,0,PC 17599,71.2833,C85,C
3,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.925,NA,S
4,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1,C123,S
5,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.05,NA,S
6,6,0,3,"Moran, Mr. James",male,NA,0,0,330877,8.4583,NA,Q


In [41]:
describe(titanic[:Survived])

Summary Stats:
Mean:           0.383838
Minimum:        0.000000
1st Quartile:   0.000000
Median:         0.000000
3rd Quartile:   1.000000
Maximum:        1.000000
Length:         891
Type:           Int64
Number Missing: 0
% Missing:      0.000000


To make things a bit more readable, let's first replace 0 and 1 with labels `Dead` and `Survived`

In [42]:
@enum SurvivedType Dead=0 Survived=1
titanic[:Survived] = SurvivedType.(titanic[:Survived]);

The `freqtable` function from the `FreqTables` package can now be used to print a nicely formatted table counting how many values are in each category

In [43]:
freqtable(titanic, :Survived)

2-element Named Array{Int64,1}
Survived  │ 
──────────┼────
Dead      │ 549
Survived  │ 342

We can also group by multiple values:

In [44]:
freqtable(titanic, :Survived, :Sex)

2×2 Named Array{Int64,2}
Survived ╲ Sex │ female    male
───────────────┼───────────────
Dead           │     81     468
Survived       │    233     109

## Dealing with missing data

If we try to make a table of the `Embarked` column we see there are missing values:

In [45]:
freqtable(titanic, :Embarked)

LoadError: [91mMethodError: Cannot `convert` an object of type DataArrays.NAtype to an object of type String
This may have arisen from a call to the constructor String(...),
since type constructors fall back to convert methods.[39m

We can see also see this by describing this column:

In [54]:
describe(titanic[:Embarked])

Summary Stats:
Length:         891
Type:           String
Number Unique:  3
Number Missing: 0
% Missing:      0.000000


There are two missing values. We can neglect these with the `dropna()` function

In [53]:
freqtable(dropna(titanic[:Embarked]))

3-element Named Array{Int64,1}
Dim1  │ 
──────┼────
C     │ 168
Q     │  77
S     │ 646

Once we've dropped the missing data, we see there are three different embarkation points: Cherbourg (C), Queenstown (Q), or Southampton (S). Southampton is the most popular (modal value), so let's fill in the missing values with that:

In [52]:
embarked_mode = mode(dropna(titanic[:Embarked]))
titanic[isna.(titanic[:, :Embarked]), :Embarked] = embarked_mode

"S"

We have a similar problem with `Age`, where 20% of the data are missing. We can simply delete those rows for now, but perhaps you can think of a better solution.

In [55]:
describe(titanic[:Age])

Summary Stats:
Mean:           29.699118
Minimum:        0.420000
1st Quartile:   20.125000
Median:         28.000000
3rd Quartile:   38.000000
Maximum:        80.000000
Length:         714
Type:           Float64
Number Missing: 177
% Missing:      19.865320


In [56]:
titanic = titanic[.!isna.(titanic[:Age]), :];

In [57]:
describe(titanic[:Age])

Summary Stats:
Mean:           29.699118
Minimum:        0.420000
1st Quartile:   20.125000
Median:         28.000000
3rd Quartile:   38.000000
Maximum:        80.000000
Length:         714
Type:           Float64
Number Missing: 0
% Missing:      0.000000


## Visualisation

Let's start digging into the data with some pie charts

In [58]:
male = titanic[titanic[:Sex] .== "male", :]
female = titanic[titanic[:Sex] .== "female", :]

pie(["Dead", "Survived"], freqtable(male, :Survived), layout=(1, 2), title="Male")
pie!(["Dead", "Survived"], freqtable(female, :Survived), layout=(1, 2), subplot=2, title="Female")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Male 
 
<polygon clip-path="url(#clip0600)" points="
146.063,223.268 266.89,223.268 266.871,221.159 266.816,219.051 266.724,216.944 266.595,214.839 266.43,212.737 266.228,210.638 265.989,208.543 265.714,206.452 
 265.402,204.366 265.054,202.286 264.67,200.213 264.249,198.146 263.793,196.088 263.301,194.037 262.773,191.995 262.209,189.963 261.61,187.941 260.976,185.93 
 260.307,183.93 259.603,181.943 258.865,179.967 258.092,178.005 257.285,176.057 256.444,174.123 255.569,172.204 254.661,170.301 253.72,168.414 252.747,166.543 
 251.74,164.69 250.702,162.854 249.632,161.037 248.53,159.239 247.397,157.461 246.233,155.702 245.038,153.964 243.814,152.248 242.56,150.552 241.276,148.879 
 239.963,147.229 238.622,145.602 237.252,143.998 235.855,142.419 234.43,140.864 232.978,139.334 231.5,137.83 229.996,136.352 228.467,134.901 226.912,133.476 
 225.332,132.079 223.729,130.709 222.102,129.368 220.451,128.055 218.778,126.771 217.083,125.517 215.366,124.292 213.628,123.098 211.87,121.934 210.091,120.801 
 208.293,119.699 206.476,118.629 204.641,117.59 202.788,116.584 200.917,115.61 199.03,114.669 197.127,113.761 195.208,112.887 193.274,112.046 191.325,111.239 
 189.363,110.466 187.388,109.728 185.4,109.024 183.401,108.355 181.389,107.721 179.367,107.122 177.335,106.558 175.294,106.03 173.243,105.538 171.184,105.081 
 169.118,104.661 167.044,104.277 164.964,103.929 162.879,103.617 160.788,103.342 158.693,103.103 156.594,102.901 154.491,102.735 152.387,102.607 150.28,102.515 
 148.172,102.459 146.063,102.441 143.954,102.459 141.846,102.515 139.739,102.607 137.635,102.735 135.532,102.901 133.433,103.103 131.338,103.342 129.247,103.617 
 127.162,103.929 125.082,104.277 123.008,104.661 120.942,105.081 118.883,105.538 116.832,106.03 114.791,106.558 112.759,107.122 110.737,107.721 108.725,108.355 
 106.726,109.024 104.738,109.728 102.763,110.466 100.8,111.239 98.8522,112.046 96.9183,112.887 94.9994,113.761 93.096,114.669 91.2088,115.61 89.3383,116.584 
 87.485,117.59 85.6496,118.629 83.8326,119.699 82.0346,120.801 80.256,121.934 78.4975,123.098 76.7596,124.292 75.0428,125.517 73.3476,126.771 71.6746,128.055 
 70.0242,129.368 68.397,130.709 66.7935,132.079 65.2141,133.476 63.6593,134.901 62.1297,136.352 60.6256,137.83 59.1475,139.334 57.6959,140.864 56.2712,142.419 
 54.8739,143.998 53.5043,145.602 52.163,147.229 50.8502,148.879 49.5664,150.552 48.3121,152.248 47.0875,153.964 45.8931,155.702 44.7291,157.461 43.5961,159.239 
 42.4942,161.037 41.4239,162.854 40.3855,164.69 39.3793,166.543 38.4056,168.414 37.4646,170.301 36.5567,172.204 35.6822,174.123 34.8414,176.057 34.0344,178.005 
 33.2615,179.967 32.523,181.943 31.819,183.93 31.1499,185.93 30.5158,187.941 29.9168,189.963 29.3533,191.995 28.8253,194.037 28.333,196.088 27.8766,198.146 
 27.4561,200.213 27.0719,202.286 26.7238,204.366 26.4121,206.452 26.1368,208.543 25.8981,210.638 25.696,212.737 25.5305,214.839 25.4018,216.944 25.3098,219.051 
 25.2546,221.159 25.2362,223.268 25.2546,225.376 25.3098,227.485 25.4018,229.591 25.5305,231.696 25.696,233.798 25.8981,235.898 26.1368,237.993 26.4121,240.084 
 26.7238,242.169 27.0719,244.249 27.4561,246.323 27.8766,248.389 28.333,250.448 28.8253,252.498 29.3533,254.54 29.9168,256.572 30.5158,258.594 31.1499,260.605 
 31.819,262.605 32.523,264.593 33.2615,266.568 34.0344,268.53 34.8414,270.478 35.6822,272.412 36.5567,274.331 37.4646,276.235 38.4056,278.122 39.3793,279.992 
 40.3855,281.846 41.4239,283.681 42.4942,285.498 43.5961,287.296 44.7291,289.075 45.8931,290.833 47.0875,292.571 48.3121,294.288 49.5664,295.983 50.8502,297.656 
 52.163,299.306 53.5043,300.934 54.8739,302.537 56.2712,304.117 57.6959,305.671 59.1475,307.201 60.6256,308.705 62.1297,310.183 63.6593,311.635 65.2141,313.06 
 66.7935,314.457 68.397,315.826 70.0242,317.168 71.6746,318.481 73.3476,319.764 75.0428,321.019 76.7596,322.243 78.4975,323.438 80.256,324.602 82.0346,325.735 
 83.8326,326.836 85.6496,327.907 

### Histogram

As before, we can use histograms to get a better picture of the distribution of a variable

In [59]:
@df titanic histogram(:Age, xlabel="Age", ylabel="Frequency", bins=20, legend=false)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 80 
 
 
 0 
 
 
 25 
 
 
 50 
 
 
 75 
 
 
 100 
 
 
 Age 
 
 
 Frequency

### Density plots

A density plot is similar to a histogram, but drawn with a line interpolating the bars. This makes it easier to overlay multiple plots, e.g. to compare the age distributions for passengers who did and did not survive.

In [60]:
@df titanic density(:Age, groups=:Survived, lw=3)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 80 
 
 
 0.005 
 
 
 0.010 
 
 
 0.015 
 
 
 0.020 
 
 
 0.025 
 
 
 0.030 
 
<polyline clip-path="url(#clip1002)" style="stroke:#009af9; stroke-width:3; stroke-opacity:1; fill:none" points="
 49.5928,375.812 52.105,375.809 54.6172,375.803 57.1294,375.793 59.6415,375.778 62.1537,375.752 64.6659,375.712 67.1781,375.651 69.6903,375.558 72.2025,375.421 
 74.7147,375.223 77.2268,374.942 79.739,374.554 82.2512,374.026 84.7634,373.327 87.2756,372.418 89.7878,371.265 92.3,369.832 94.8121,368.093 97.3243,366.03 
 99.8365,363.637 102.349,360.928 104.861,357.933 107.373,354.701 109.885,351.299 112.397,347.807 114.91,344.312 117.422,340.902 119.934,337.662 122.446,334.659 
 124.958,331.942 127.471,329.54 129.983,327.454 132.495,325.666 135.007,324.138 137.519,322.82 140.031,321.654 142.544,320.58 145.056,319.538 147.568,318.47 
 150.08,317.316 152.592,316.01 155.105,314.474 157.617,312.612 160.129,310.304 162.641,307.407 165.153,303.752 167.666,299.156 170.178,293.427 172.69,286.386 
 175.202,277.873 177.714,267.775 180.226,256.034 182.739,242.664 185.251,227.756 187.763,211.483 190.275,194.09 192.787,175.891 195.3,157.245 197.812,138.544 
 200.324,120.185 202.836,102.554 205.348,85.9993 207.86,70.8199 210.373,57.2493 212.885,45.4489 215.397,35.5056 217.909,27.4343 220.421,21.1853 222.934,16.6547 
 225.446,13.6979 227.958,12.1442 230.47,11.811 232.982,12.518 235.495,14.0985 238.007,16.4088 240.519,19.3345 243.031,22.7924 245.543,26.7302 248.055,31.1223 
 250.568,35.9626 253.08,41.2568 255.592,47.0129 258.104,53.2326 260.616,59.9042 263.129,66.998 265.641,74.464 268.153,82.2334 270.665,90.2217 273.177,98.3345 
 275.689,106.474 278.202,114.547 280.714,122.467 283.226,130.164 285.738,137.582 288.25,144.684 290.763,151.449 293.275,157.869 295.787,163.951 298.299,169.706 
 300.811,175.153 303.324,180.313 305.836,185.207 308.348,189.858 310.86,194.285 313.372,198.509 315.884,202.553 318.397,206.441 320.909,210.2 323.421,213.861 
 325.933,217.46 328.445,221.035 330.958,224.627 333.47,228.272 335.982,232.004 338.494,235.85 341.006,239.825 343.518,243.936 346.031,248.174 348.543,252.523 
 351.055,256.955 353.567,261.436 356.079,265.927 358.592,270.387 361.104,274.776 363.616,279.057 366.128,283.196 368.64,287.165 371.153,290.94 373.665,294.503 
 376.177,297.841 378.689,300.947 381.201,303.817 383.713,306.455 386.226,308.869 388.738,311.071 391.25,313.079 393.762,314.914 396.274,316.601 398.787,318.167 
 401.299,319.642 403.811,321.056 406.323,322.438 408.835,323.815 411.348,325.21 413.86,326.643 416.372,328.128 418.884,329.673 421.396,331.279 423.908,332.94 
 426.421,334.644 428.933,336.374 431.445,338.108 433.957,339.819 436.469,341.484 438.982,343.08 441.494,344.589 444.006,346 446.518,347.311 449.03,348.531 
 451.542,349.674 454.055,350.766 456.567,351.835 459.079,352.91 461.591,354.02 464.103,355.187 466.616,356.426 469.128,357.741 471.64,359.129 474.152,360.574 
 476.664,362.054 479.177,363.544 481.689,365.013 484.201,366.434 486.713,367.779 489.225,369.028 491.737,370.164 494.25,371.178 496.762,372.065 499.274,372.827 
 501.786,373.47 504.298,374.002 506.811,374.435 509.323,374.78 511.835,375.052 514.347,375.261 516.859,375.419 519.371,375.537 521.884,375.623 524.396,375.685 
 526.908,375.729 529.42,375.759 531.932,375.779 534.445,375.793 536.957,375.802 539.469,375.807 541.981,375.811 544.493,375.813 547.006,375.813 549.518,375.813 
 
 "/>
<polyline clip-path="url(#clip1002)" style="stroke:#e26f46; stroke-width:3; stroke-opacity:1; fill:none" points="
 45.4014,375.804 48.0894,375.794 50.7774,375.777 53.4654,375.747 56.1534,375.698 58.8414,375.616 61.5294,375.487 64.2174,375.285 66.9054,374.978 69.5934,374.52 
 72.2814,373.853 74.9695,372.905 77.6575,371.587 80.3455,369.798 83.0335,367.425 85.7215,364.352 88.4095,360.465 91.0975,355.665 93.7855,349.881 96.4735,343.083 
 99.1615,335.

In [61]:
@df titanic density(:Age, groups=:Sex, lw=3)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 80 
 
 
 0.005 
 
 
 0.010 
 
 
 0.015 
 
 
 0.020 
 
 
 0.025 
 
 
 0.030 
 
<polyline clip-path="url(#clip1202)" style="stroke:#009af9; stroke-width:3; stroke-opacity:1; fill:none" points="
 45.4014,375.807 47.7346,375.803 50.0679,375.796 52.4012,375.784 54.7345,375.767 57.0677,375.741 59.401,375.703 61.7343,375.647 64.0675,375.568 66.4008,375.456 
 68.7341,375.301 71.0674,375.088 73.4006,374.799 75.7339,374.414 78.0672,373.906 80.4004,373.245 82.7337,372.398 85.067,371.327 87.4003,369.991 89.7335,368.347 
 92.0668,366.352 94.4001,363.966 96.7333,361.151 99.0666,357.877 101.4,354.125 103.733,349.886 106.066,345.169 108.4,339.998 110.733,334.416 113.066,328.485 
 115.4,322.285 117.733,315.912 120.066,309.475 122.399,303.093 124.733,296.887 127.066,290.979 129.399,285.483 131.732,280.497 134.066,276.104 136.399,272.362 
 138.732,269.303 141.065,266.929 143.399,265.213 145.732,264.099 148.065,263.502 150.399,263.312 152.732,263.398 155.065,263.615 157.398,263.803 159.732,263.802 
 162.065,263.447 164.398,262.585 166.731,261.07 169.065,258.778 171.398,255.606 173.731,251.475 176.065,246.339 178.398,240.183 180.731,233.023 183.064,224.909 
 185.398,215.92 187.731,206.162 190.064,195.765 192.397,184.876 194.731,173.651 197.064,162.255 199.397,150.849 201.731,139.589 204.064,128.62 206.397,118.07 
 208.73,108.053 211.064,98.66 213.397,89.967 215.73,82.0297 218.063,74.8873 220.397,68.5634 222.73,63.0683 225.063,58.4002 227.397,54.5466 229.73,51.4857 
 232.063,49.1875 234.396,47.6145 236.73,46.7229 239.063,46.4641 241.396,46.7861 243.729,47.635 246.063,48.9577 248.396,50.7035 250.729,52.8259 253.063,55.2842 
 255.396,58.0449 257.729,61.0813 260.062,64.3735 262.396,67.9073 264.729,71.6726 267.062,75.6617 269.395,79.867 271.729,84.2797 274.062,88.8884 276.395,93.678 
 278.729,98.6301 281.062,103.723 283.395,108.932 285.728,114.234 288.062,119.602 290.395,125.016 292.728,130.456 295.061,135.905 297.395,141.354 299.728,146.796 
 302.061,152.227 304.394,157.647 306.728,163.056 309.061,168.455 311.394,173.844 313.728,179.218 316.061,184.572 318.394,189.892 320.727,195.165 323.061,200.372 
 325.394,205.49 327.727,210.498 330.06,215.373 332.394,220.094 334.727,224.644 337.06,229.011 339.394,233.187 341.727,237.173 344.06,240.977 346.393,244.61 
 348.727,248.093 351.06,251.448 353.393,254.701 355.726,257.879 358.06,261.008 360.393,264.114 362.726,267.216 365.06,270.33 367.393,273.468 369.726,276.633 
 372.059,279.827 374.393,283.046 376.726,286.28 379.059,289.52 381.392,292.753 383.726,295.967 386.059,299.15 388.392,302.293 390.726,305.387 393.059,308.425 
 395.392,311.406 397.725,314.327 400.059,317.19 402.392,319.998 404.725,322.754 407.058,325.463 409.392,328.13 411.725,330.759 414.058,333.356 416.392,335.922 
 418.725,338.46 421.058,340.969 423.391,343.447 425.725,345.889 428.058,348.29 430.391,350.641 432.724,352.933 435.058,355.153 437.391,357.29 439.724,359.331 
 442.058,361.265 444.391,363.08 446.724,364.766 449.057,366.318 451.391,367.73 453.724,368.999 456.057,370.128 458.39,371.12 460.724,371.98 463.057,372.718 
 465.39,373.341 467.723,373.861 470.057,374.29 472.39,374.639 474.723,374.919 477.057,375.14 479.39,375.313 481.723,375.447 484.056,375.548 486.39,375.624 
 488.723,375.68 491.056,375.721 493.389,375.751 495.723,375.772 498.056,375.786 500.389,375.796 502.723,375.803 505.056,375.807 507.389,375.808 509.722,375.809 
 
 "/>
<polyline clip-path="url(#clip1202)" style="stroke:#e26f46; stroke-width:3; stroke-opacity:1; fill:none" points="
 55.7449,375.81 58.381,375.804 61.017,375.794 63.653,375.776 66.289,375.746 68.9251,375.696 71.5611,375.615 74.1971,375.488 76.8332,375.294 79.4692,375.002 
 82.1052,374.576 84.7413,373.967 87.3773,373.121 90.0133,371.971 92.6493,370.45 95.2854,368.487 97.9214,366.018 100.557,362.994 103.193,359.388 105.829,355.204 
 108.466,350.485 111

## Feature Engineering

Let's make a new column `Child` for people under 13, based on the bump in age distrubtions

In [62]:
@enum ChildType Child=0 Adult=1

function classify_by_age(x)
  if x < 13
    Child
  else
    Adult
  end
end

titanic[:Child] = classify_by_age.(titanic[:Age]);

In [63]:
freqtable(titanic, :Child, :Survived)

2×2 Named Array{Int64,2}
Child ╲ Survived │     Dead  Survived
─────────────────┼───────────────────
Child            │       29        40
Adult            │      395       250

This feature could be used as one of the inputs into a decision tree or other predictive model (cf. bonus question in the assignment!)